In [263]:
import glob
# This file should run basically insta fast. Bad coding for sure 
# There is a place where i sometimes comment out lin/rbf/sigmoid stuff
# Dont do that, it really helps for some subjects. 
# Anything. Leave only linear. For fast testing of best features
# But dont do that either. 

genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "11" # Which month and days result you want to visualize in table
month = 11

repetitionValue = 51
repetitionName =   "testJ3"  # "udrliplotnoAda3"  #  "udrlBC4CVTest" # 
#  "udrliplotnoAda3hyperparams" 
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

saveTop = True
if saveTop:
    saveTopName = "testp1rli.npy"
else:
    saveTopName = None
topAmount = 20
skipSubjects = []
onlySignificant = True
minAcc = 0.1
print(len(savedResults))
print(len(savedResults)/9)

4
0.4444444444444444


In [264]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                
                
                # if "lin" in akernel:
                #     continue
                # if "sig" in akernel:
                #     continue
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
nameArray = np.array(list(nameDict))
topPerFeature = np.zeros([10, len(nameDict)])
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]
            
            comboName = list(nameDict.keys())[combinationList[res[0]][0]]
            nameIndex = np.where(nameArray == comboName)[0][0]
            if res[1] > topPerFeature[subNr, nameIndex]:
                topPerFeature[subNr, nameIndex] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 630)
(10, 630)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

In [265]:
x = 0

pd.set_option("display.max_columns", None)
print()
nameDictNrs = np.ones([1, len(nameDict)])
nameDictNrs[0,:] = np.arange(len(nameDict)) * 3
print(nameDictNrs.shape)
#table = pd.DataFrame(nameDictNrs , columns=nameDict.keys())
table = pd.DataFrame(topPerFeature, columns=nameDict.keys())
#display(table)
display(table)
# for names in nameDict.keys():
#     print(names, end="        ")
#     x +=1
#     #print(x)



(1, 210)


,-fftData,-welchData,-dataHR,-dataCorr1d,-dataFFTCV-BC,-dataWCV-BC,-dataHRCV-BC,-fftDataBC,-welchDataBC,-dataHRBC,-gaussianData,-dataGCVBC,-gaussianDataBC,-dataCorr1dBC,-inverseFFT-BC,-dataCorr1d01s,-dataCorr1d02s,-iFFTdataCorr1d01s-BC,-iFFTdataCorr1d02s-BC,-iFFTdataCorr1d005s-BC,welchData--fftData,dataHR--fftData,dataCorr1d--fftData,dataFFTCV-BC--fftData,dataWCV-BC--fftData,dataHRCV-BC--fftData,fftDataBC--fftData,welchDataBC--fftData,dataHRBC--fftData,gaussianData--fftData,dataGCVBC--fftData,gaussianDataBC--fftData,dataCorr1dBC--fftData,inverseFFT-BC--fftData,dataCorr1d01s--fftData,dataCorr1d02s--fftData,iFFTdataCorr1d01s-BC--fftData,iFFTdataCorr1d02s-BC--fftData,iFFTdataCorr1d005s-BC--fftData,dataHR--welchData,dataCorr1d--welchData,dataFFTCV-BC--welchData,dataWCV-BC--welchData,dataHRCV-BC--welchData,fftDataBC--welchData,welchDataBC--welchData,dataHRBC--welchData,gaussianData--welchData,dataGCVBC--welchData,gaussianDataBC--welchData,dataCorr1dBC--welchData,inverseFFT-BC--welchData,dataCorr1d01s--welchData,dataCorr1d02s--welchData,iFFTdataCorr1d01s-BC--welchData,iFFTdataCorr1d02s-BC--welchData,iFFTdataCorr1d005s-BC--welchData,dataCorr1d--dataHR,dataFFTCV-BC--dataHR,dataWCV-BC--dataHR,dataHRCV-BC--dataHR,fftDataBC--dataHR,welchDataBC--dataHR,dataHRBC--dataHR,gaussianData--dataHR,dataGCVBC--dataHR,gaussianDataBC--dataHR,dataCorr1dBC--dataHR,inverseFFT-BC--dataHR,dataCorr1d01s--dataHR,dataCorr1d02s--dataHR,iFFTdataCorr1d01s-BC--dataHR,iFFTdataCorr1d02s-BC--dataHR,iFFTdataCorr1d005s-BC--dataHR,dataFFTCV-BC--dataCorr1d,dataWCV-BC--dataCorr1d,dataHRCV-BC--dataCorr1d,fftDataBC--dataCorr1d,welchDataBC--dataCorr1d,dataHRBC--dataCorr1d,gaussianData--dataCorr1d,dataGCVBC--dataCorr1d,gaussianDataBC--dataCorr1d,dataCorr1dBC--dataCorr1d,inverseFFT-BC--dataCorr1d,dataCorr1d01s--dataCorr1d,dataCorr1d02s--dataCorr1d,iFFTdataCorr1d01s-BC--dataCorr1d,iFFTdataCorr1d02s-BC--dataCorr1d,iFFTdataCorr1d005s-BC--dataCorr1d,dataWCV-BC--dataFFTCV-BC,dataHRCV-BC--dataFFTCV-BC,fftDataBC--dataFFTCV-BC,welchDataBC--dataFFTCV-BC,dataHRBC--dataFFTCV-BC,gaussianData--dataFFTCV-BC,dataGCVBC--dataFFTCV-BC,gaussianDataBC--dataFFTCV-BC,dataCorr1dBC--dataFFTCV-BC,inverseFFT-BC--dataFFTCV-BC,dataCorr1d01s--dataFFTCV-BC,dataCorr1d02s--dataFFTCV-BC,iFFTdataCorr1d01s-BC--dataFFTCV-BC,iFFTdataCorr1d02s-BC--dataFFTCV-BC,iFFTdataCorr1d005s-BC--dataFFTCV-BC,dataHRCV-BC--dataWCV-BC,fftDataBC--dataWCV-BC,welchDataBC--dataWCV-BC,dataHRBC--dataWCV-BC,gaussianData--dataWCV-BC,dataGCVBC--dataWCV-BC,gaussianDataBC--dataWCV-BC,dataCorr1dBC--dataWCV-BC,inverseFFT-BC--dataWCV-BC,dataCorr1d01s--dataWCV-BC,dataCorr1d02s--dataWCV-BC,iFFTdataCorr1d01s-BC--dataWCV-BC,iFFTdataCorr1d02s-BC--dataWCV-BC,iFFTdataCorr1d005s-BC--dataWCV-BC,fftDataBC--dataHRCV-BC,welchDataBC--dataHRCV-BC,dataHRBC--dataHRCV-BC,gaussianData--dataHRCV-BC,dataGCVBC--dataHRCV-BC,gaussianDataBC--dataHRCV-BC,dataCorr1dBC--dataHRCV-BC,inverseFFT-BC--dataHRCV-BC,dataCorr1d01s--dataHRCV-BC,dataCorr1d02s--dataHRCV-BC,iFFTdataCorr1d01s-BC--dataHRCV-BC,iFFTdataCorr1d02s-BC--dataHRCV-BC,iFFTdataCorr1d005s-BC--dataHRCV-BC,welchDataBC--fftDataBC,dataHRBC--fftDataBC,gaussianData--fftDataBC,dataGCVBC--fftDataBC,gaussianDataBC--fftDataBC,dataCorr1dBC--fftDataBC,inverseFFT-BC--fftDataBC,dataCorr1d01s--fftDataBC,dataCorr1d02s--fftDataBC,iFFTdataCorr1d01s-BC--fftDataBC,iFFTdataCorr1d02s-BC--fftDataBC,iFFTdataCorr1d005s-BC--fftDataBC,dataHRBC--welchDataBC,gaussianData--welchDataBC,dataGCVBC--welchDataBC,gaussianDataBC--welchDataBC,dataCorr1dBC--welchDataBC,inverseFFT-BC--welchDataBC,dataCorr1d01s--welchDataBC,dataCorr1d02s--welchDataBC,iFFTdataCorr1d01s-BC--welchDataBC,iFFTdataCorr1d02s-BC--welchDataBC,iFFTdataCorr1d005s-BC--welchDataBC,gaussianData--dataHRBC,dataGCVBC--dataHRBC,gaussianDataBC--dataHRBC,dataCorr1dBC--dataHRBC,inverseFFT-BC--dataHRBC,dataCorr1d01s--dataHRBC,dataCorr1d02s--dataHRBC,iFFTdataCorr1d01s-BC--dataHRBC,iFFTdataCorr1d02s-BC--dataHRBC,iFFTdataCorr1d005s-BC--dataHRBC,dataGCVBC--gaussianData,gaussianDataBC--gaussia

In [266]:
import matplotlib.pyplot as plt
# ordeTopAverage = np.array(np.argsort(comboAverage, axis=1)/(len(combinationList))*len(dataF),dtype=int)
# print(ordeTopAverage)
# print(len(combinationList))
# print(len(dataF))
# print(len(topPerFeature))
# print(len(combinationList)/len(dataF))
#print(ordeTopAverage)

#np.sort(topPerFeature, axis=1)
#np.argsort(comboAverage, axis=1)
#sortA = np.argsort(topPerFeature, axis = 1)
if len(nameArray) < 200:
    plt.figure(figsize=[30,10])
    plt.boxplot(topPerFeature[1:])
    plt.legend(nameArray)
    print("")
# else: 
#     plt.figure(figsize=[30,10])
#     fig, ax = plt.subplots(figsize=[30,10])
#     plt.margins(y=1.5)
#     ax.set_ylim([np.min(topPerFeature[1:,ordeTopAverage[0][:topAmount]])-0.3,1])
#     plt.boxplot(topPerFeature[1:,ordeTopAverage[0][:topAmount]])
#     plt.legend(nameArray[ordeTopAverage[0][-topAmount:]])
#     plt.title(f"Best {topAmount} feature combos by average accuracy over subjects")
#     print(nameArray[ordeTopAverage[0][-topAmount:]])
    


In [267]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.99 : subject 1, feature:welchDataBC--welchData, kernellinear, C:2.5
Accuracy 0.99 : subject 1, feature:welchDataBC--fftData, kernelsigmoid, C:2.5
Accuracy 0.99 : subject 1, feature:welchDataBC--fftData, kernelrbf, C:2.5
Accuracy 0.99 : subject 1, feature:welchDataBC--fftData, kernellinear, C:2.5
Accuracy 0.99 : subject 1, feature:fftDataBC--welchData, kernellinear, C:2.5
Accuracy 0.99 : subject 1, feature:dataGCVBC--welchData, kernelrbf, C:2.5
Accuracy 0.99 : subject 1, feature:dataGCVBC--welchData, kernellinear, C:2.5
Accuracy 0.99 : subject 1, feature:-welchDataBC, kernelrbf, C:2.5
Accuracy 0.99 : subject 1, feature:-welchDataBC, kernellinear, C:2.5
Accuracy 0.99 : subject 1, feature:-dataGCVBC, kernellinear, C:2.5


In [268]:
# top for each feature

In [269]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
arrayOfTopFeatureNames = []
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop[:1]:
        arrayOfTopFeatureNames.append(top[2])
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )
#print(arrayOfTopFeatureNames)
arrayOfTopFeatureNames = np.array(arrayOfTopFeatureNames, dtype=object)
#print(np.array(arrayOfTopFeatureNames, dtype=object).shape)
newArrayOfTopFeatureNames = []
for topNames in arrayOfTopFeatureNames:
    temp = topNames.replace("--","-")
    temp2 = temp.replace("-B","%")
    splitArray = np.array(temp2.split("-"),dtype=object)
    for ind, stri in enumerate(splitArray):
        splitArray[ind] = stri.replace("%","-B")
    skip = False  
    for obj in splitArray:
        #print(obj)
        if obj == "":
            #print("aofafa")
            skip = True
    if skip:
        continue
    #print(splitArray)
    newArrayOfTopFeatureNames.append(splitArray)
    
#print(newArrayOfTopFeatureNames)
newArrayOfTopFeatureNames = np.array(newArrayOfTopFeatureNames, dtype=object) 
#saveTopName = "top2udrli"
if saveTopName is not None:
    #print(newArrayOfTopFeatureNames)
    np.save(f"topFeatures/{saveTopName}", newArrayOfTopFeatureNames)
    print("TopFeatureNameArraySaved")
    print(newArrayOfTopFeatureNames)


Top ten highest accuracies

Accuracy 0.99 : subject 4, feature:-fftDataBC, kernelsigmoid, C:2.5

Accuracy 0.99 : subject 3, feature:dataHRBC--dataHR, kernelsigmoid, C:2.5

Accuracy 0.99 : subject 2, feature:-dataWCV-BC, kernelrbf, C:2.5

Accuracy 0.99 : subject 1, feature:-welchDataBC, kernelrbf, C:2.5
TopFeatureNameArraySaved
[['dataHRBC' 'dataHR']]


In [270]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
# print(headers[maxInd])
# print(nrOfSubjects[maxInd])
# print(list(nameDict.keys())[combinationList[maxInd][0]])
# print(list(kernelDict.keys())[combinationList[maxInd][1]])
# print(f"C: {combinationList[maxInd][2]}")


[0.         0.7694127  0.70872751 0.72123633 0.78638889 0.
 0.         0.         0.         0.        ]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629
0,0.536806,0.513889,0.505556,0.560417,0.494444,0.481944,0.469444,0.399306,0.470139,0.497917,0.522222,0.507639,0.904167,0.841667,0.876389,0.9225,0.758333,0.913472,0.739583,0.666667,0.715278,0.960694,0.960694,0.946806,0.942917,0.932917,0.929028,0.873611,0.836806,0.823611,0.496528,0.499306,0.528472,0.886389,0.872917,0.897222,0.88125,0.854861,0.89375,0.858333,0.877778,0.859028,0.45625,0.445833,0.747222,0.517361,0.471528,0.507639,0.572222,0.526389,0.523611,0.384722,0.429861,0.714583,0.49375,0.445833,0.765278,0.474306,0.518056,0.731944,0.568056,0.5375,0.524306,0.465972,0.444444,0.48125,0.486111,0.4875,0.520139,0.89125,0.848611,0.880833,0.536806,0.513889,0.503472,0.679861,0.681944,0.719444,0.960694,0.960694,0.940278,0.956806,0.970694,0.960278,0.831944,0.847917,0.748611,0.461806,0.447222,0.467361,0.960694,0.946806,0.968611,0.884306,0.867361,0.883333,0.904167,0.88125,0.872917,0.539583,0.463194,0.741667,0.465278,0.459722,0.459028,0.485417,0.470833,0.501389,0.469444,0.461111,0.722917,0.511111,0.490278,0.761806,0.554167,0.5125,0.745833,0.475,0.497222,0.482639,0.528472,0.480556,0.505556,0.911111,0.801389,0.886806,0.871389,0.827778,0.835694,0.725,0.639583,0.738194,0.970694,0.934306,0.946806,0.942917,0.91,0.919028,0.821528,0.809028,0.80625,0.459028,0.466667,0.442361,0.942917,0.932917,0.942917,0.894306,0.865278,0.883333,0.917639,0.869444,0.869444,0.561111,0.54375,0.703472,0.486111,0.452083,0.423611,0.523611,0.502083,0.4875,0.515278,0.504167,0.665278,0.526389,0.498611,0.713889,0.53125,0.554167,0.689583,0.486806,0.472917,0.463194,0.924583,0.786111,0.859028,0.5875,0.525,0.549306,0.

Max average accuracy 0.9706944444444445


In [271]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboAverage.argsort()[0][-5:][::-1]]
top5Vals = comboAverage[0][comboAverage.argsort()[0][-5:][::-1]]
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

welchDataBC--fftData
sigmoid
0.9706944444444445

fftDataBC--welchData
linear
0.9706944444444445

fftDataBC--dataHR
sigmoid
0.9686111111111111

fftDataBC--dataFFTCV-BC
linear
0.9686111111111111

dataGCVBC--fftDataBC
rbf
0.9686111111111111



In [272]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboTotal.argsort()[0][-5:][::-1]]
top5Vals = comboTotal[0][comboTotal.argsort()[0][-5:][::-1]] /9
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

welchDataBC--fftData
sigmoid
0.4314197530864198

fftDataBC--welchData
linear
0.4314197530864198

fftDataBC--dataHR
sigmoid
0.43049382716049384

fftDataBC--dataFFTCV-BC
linear
0.43049382716049384

dataGCVBC--fftDataBC
rbf
0.43049382716049384



In [273]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629
0,2.147222,2.055556,2.022222,2.241667,1.977778,1.927778,1.877778,1.597222,1.880556,1.991667,2.088889,2.030556,3.616667,3.366667,3.505556,3.69,3.033333,3.653889,2.958333,2.666667,2.861111,3.842778,3.842778,3.787222,3.771667,3.731667,3.716111,3.494444,3.347222,3.294444,1.986111,1.997222,2.113889,3.545556,3.491667,3.588889,3.525,3.419444,3.575,3.433333,3.511111,3.436111,1.825,1.783333,2.988889,2.069444,1.886111,2.030556,2.288889,2.105556,2.094444,1.538889,1.719444,2.858333,1.975,1.783333,3.061111,1.897222,2.072222,2.927778,2.272222,2.15,2.097222,1.863889,1.777778,1.925,1.944444,1.95,2.080556,3.565,3.394444,3.523333,2.147222,2.055556,2.013889,2.719444,2.727778,2.877778,3.842778,3.842778,3.761111,3.827222,3.882778,3.841111,3.327778,3.391667,2.994444,1.847222,1.788889,1.869444,3.842778,3.787222,3.874444,3.537222,3.469444,3.533333,3.616667,3.525,3.491667,2.158333,1.852778,2.966667,1.861111,1.838889,1.836111,1.941667,1.883333,2.005556,1.877778,1.844444,2.891667,2.044444,1.961111,3.047222,2.216667,2.05,2.983333,1.9,1.988889,1.930556,2.113889,1.922222,2.022222,3.644444,3.205556,3.547222,3.485556,3.311111,3.342778,2.9,2.558333,2.952778,3.882778,3.737222,3.787222,3.771667,3.64,3.676111,3.286111,3.236111,3.225,1.836111,1.866667,1.769444,3.771667,3.731667,3.771667,3.577222,3.461111,3.533333,3.670556,3.477778,3.477778,2.244444,2.175,2.813889,1.944444,1.808333,1.694444,2.094444,2.008333,1.95,2.061111,2.016667,2.661111,2.105556,1.994444,2.855556,2.125,2.216667,2.758333,1.947222,1.891667,1.852778,3.698333,3.144444,3.436111,2.35,2.1,2.197222,2.872222,2.519444,2.958333,3.842778,3.87

82
4.0
Max total significant accuracy then averaged 0.9706944444444445 
welchDataBC--fftData
sigmoid
C: 2.5 
132
4.0
Max total significant accuracy then averaged 0.9706944444444445 
fftDataBC--welchData
linear
C: 2.5 


In [274]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
